In [1]:
# !pip install opencv-python
# !pip install imagehash
# !pip install Pillow

функция хэширования видео

In [32]:
import cv2
from PIL import Image
import imagehash
import numpy as np

def get_video_hashes(video_path, frame_interval=30):
    hashes = []
    cap = cv2.VideoCapture(video_path)
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps if fps else 0
    #print(f"Обработка '{video_path}': {frame_count} кадров, {duration:.2f} секунд, {fps:.2f} FPS")
    
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            phash = imagehash.phash(pil_img)
            hashes.append(phash)
        frame_idx += 1
    cap.release()
    return hashes

def hamming_distance(hashes_video1, hashes_video2):
    num_hashes = min(len(hashes_video1), len(hashes_video2))
    distances = []
    for i in range(num_hashes):
        distance = hashes_video1[i] - hashes_video2[i] 
        distances.append(distance)
    average_distance = np.mean(distances)
    return average_distance

In [4]:
import pandas as pd
test = pd.read_csv('test.csv')

In [34]:
len(test)

500

In [221]:
dict_ = test[test['is_duplicate']==True][['uuid','duplicate_for']].set_index('uuid')['duplicate_for'].to_dict()
dict_

{'63b899be-9547-44c2-91b2-e1b71510a614': '4a14964b-3ccf-4673-93fc-86c37a489333',
 '05d72fc9-89a3-47bf-bb1b-6db4fc6f2b56': '090cb968-02eb-4aba-8b94-f6124050df59',
 '4e13f784-dc74-4532-b944-1789b3a95af1': '0e0bc479-6a06-4901-8e71-99463edc5e52',
 'b7defd6d-f2ba-4dec-b341-0a04f471a721': '1454c25b-fe4c-4073-a73a-9fba3fb048c2',
 '1ad0536e-8422-409f-9e4a-3dc64f5e260a': '3b447013-9202-415d-ae15-b1d76ce8e060',
 'ddb2778d-7c4e-4f50-9c5a-880b7d77b08b': '0409cf4d-acf5-48b0-ab20-92e58d0ba023',
 'f7b9fc3c-f6fc-4102-90c7-33361763333f': '182901f0-3e62-4d1d-a2e1-22e092db7992',
 'c318bca6-515c-4c33-961c-4c42a42d448d': '1cf2b2b7-732b-403a-aae0-61b6fd328fad',
 '390f73b5-6e10-4db6-978d-03cf06d2cc39': '2cf8f595-acd3-489f-b4ca-86d02c4e1eb2',
 'b5699703-e75a-4ea4-a8f0-1327108be3cb': '3b6af836-ccfd-4dfa-83b1-937f4f62e8a1',
 'd7009e84-c4fd-4591-abcb-124332e1746d': '57ea9230-51b6-4d6f-baaf-ee06c0d78318',
 '4904f2e0-0876-4f25-bba3-97806456bd31': '3739da22-e526-43b7-a5d5-673b905f0908',
 '21a93966-f283-4389-a818-a7

In [28]:
scores = []
for duplicate, duplicate_for in dict_.items():
    hashes_video1 = get_video_hashes(f'train_dataset/{duplicate}.mp4', frame_interval=30)
    hashes_video2 = get_video_hashes(f'train_dataset/{duplicate_for}.mp4', frame_interval=30)
    average_distance = hamming_distance(hashes_video1, hashes_video2)
    scores.append(average_distance)

Обработка 'train_dataset/63b899be-9547-44c2-91b2-e1b71510a614.mp4': 717 кадров, 23.90 секунд, 30.00 FPS
Обработка 'train_dataset/4a14964b-3ccf-4673-93fc-86c37a489333.mp4': 717 кадров, 23.90 секунд, 30.00 FPS
Обработка 'train_dataset/05d72fc9-89a3-47bf-bb1b-6db4fc6f2b56.mp4': 547 кадров, 18.23 секунд, 30.00 FPS
Обработка 'train_dataset/090cb968-02eb-4aba-8b94-f6124050df59.mp4': 547 кадров, 18.23 секунд, 30.00 FPS
Обработка 'train_dataset/4e13f784-dc74-4532-b944-1789b3a95af1.mp4': 153 кадров, 5.10 секунд, 30.00 FPS
Обработка 'train_dataset/0e0bc479-6a06-4901-8e71-99463edc5e52.mp4': 152 кадров, 5.07 секунд, 30.00 FPS
Обработка 'train_dataset/b7defd6d-f2ba-4dec-b341-0a04f471a721.mp4': 318 кадров, 10.60 секунд, 30.00 FPS
Обработка 'train_dataset/1454c25b-fe4c-4073-a73a-9fba3fb048c2.mp4': 318 кадров, 10.60 секунд, 30.00 FPS
Обработка 'train_dataset/1ad0536e-8422-409f-9e4a-3dc64f5e260a.mp4': 321 кадров, 10.70 секунд, 30.00 FPS
Обработка 'train_dataset/3b447013-9202-415d-ae15-b1d76ce8e060.mp4'

In [43]:
from sklearn.metrics import f1_score
threshold = 5
true_labels = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
pred_labels = [1 if dist <= threshold else 0 for dist in scores]
f1 = f1_score(true_labels, pred_labels)
f1

0.375

cемпл

In [50]:
videos = test['uuid'].unique()

In [90]:
import itertools
video_pairs_itertools = tuple(itertools.combinations(videos, 2))

In [94]:
len(video_pairs_itertools)

124750

In [96]:
import random
random_video_pairs_itertools = random.sample(video_pairs_itertools, 500)

In [232]:
sample = pd.DataFrame(random_video_pairs_itertools, columns = [['uuid', 'video']])

In [233]:
sample = sample['uuid'].drop_duplicates().merge(sample, left_index=True, right_index=True)
sample = sample.drop('uuid_x',axis=1)

In [234]:
container=[]
for index, row in sample.iterrows():
    container.append(row['uuid_y'])


In [235]:
is_container=[]
for uuid in container:
    value = test[test['uuid']==uuid]['is_duplicate'].values[0]
    is_container.append(value)

In [237]:
sample['is_duplicate'] = is_container
sample.columns = ['uuid', 'is_duplicate_for', 'is_duplicate']
sample = sample.reset_index(drop=True)
sample

,uuid,is_duplicate_for,is_duplicate
0,4c224695-eaff-4c1c-ada4-9040114592f0,44f636af-3b5e-43a3-ba95-3cb3671d4245,False
1,422aea2f-bdce-4205-9b7a-0a88bfc54138,0d8b8c06-6734-4679-a471-9394323137b9,False
2,346f1bce-7d25-4e70-bf10-a267b33ab345,0eb422ec-9c3f-48c8-a241-66b81b6e3ac3,False
3,0e3e3a98-a4f4-4647-a38e-9e16b5d1d0a6,0338a829-aaea-41cc-8e55-3b0a7442a423,False
4,311ef0f0-47aa-4de8-aa88-12bbdfdb8ea7,1ad0536e-8422-409f-9e4a-3dc64f5e260a,False
...,...,...,...
281,3663e734-1ad1-44f0-af93-b4af86d7011f,43819960-755f-44d3-b9e6-209a15e01f84,False
282,18896920-bbec-499f-b8f9-8cb51312153a,4fee3060-c282-41cc-ab92-67f41881951d,False
283,09b97606-c5e6-4b3f-b416-b753540fbd91,299acf23-d67d-4a85-a5cf-0ac77cf5eea6,False
284,32ef9826-4f89-4390-a356-62882893fd81,0a1d79f3-93e2-4ac0-8976-be89e71e456d,False


In [238]:
sample_add = pd.DataFrame(dict_.items(), columns = ['uuid', 'is_duplicate_for'])
sample_add['is_duplicate']= True
sample_add

,uuid,is_duplicate_for,is_duplicate
0,63b899be-9547-44c2-91b2-e1b71510a614,4a14964b-3ccf-4673-93fc-86c37a489333,True
1,05d72fc9-89a3-47bf-bb1b-6db4fc6f2b56,090cb968-02eb-4aba-8b94-f6124050df59,True
2,4e13f784-dc74-4532-b944-1789b3a95af1,0e0bc479-6a06-4901-8e71-99463edc5e52,True
3,b7defd6d-f2ba-4dec-b341-0a04f471a721,1454c25b-fe4c-4073-a73a-9fba3fb048c2,True
4,1ad0536e-8422-409f-9e4a-3dc64f5e260a,3b447013-9202-415d-ae15-b1d76ce8e060,True
5,ddb2778d-7c4e-4f50-9c5a-880b7d77b08b,0409cf4d-acf5-48b0-ab20-92e58d0ba023,True
6,f7b9fc3c-f6fc-4102-90c7-33361763333f,182901f0-3e62-4d1d-a2e1-22e092db7992,True
7,c318bca6-515c-4c33-961c-4c42a42d448d,1cf2b2b7-732b-403a-aae0-61b6fd328fad,True
8,390f73b5-6e10-4db6-978d-03cf06d2cc39,2cf8f595-acd3-489f-b4ca-86d02c4e1eb2,True
9,b5699703-e75a-4ea4-a8f0-1327108be3cb,3b6af836-ccfd-4dfa-83b1-937f4f62e8a1,True


In [240]:
sample = pd.concat([sample, sample_add])
sample = sample.reset_index(drop=True)
sample

,uuid,is_duplicate_for,is_duplicate
0,4c224695-eaff-4c1c-ada4-9040114592f0,44f636af-3b5e-43a3-ba95-3cb3671d4245,False
1,422aea2f-bdce-4205-9b7a-0a88bfc54138,0d8b8c06-6734-4679-a471-9394323137b9,False
2,346f1bce-7d25-4e70-bf10-a267b33ab345,0eb422ec-9c3f-48c8-a241-66b81b6e3ac3,False
3,0e3e3a98-a4f4-4647-a38e-9e16b5d1d0a6,0338a829-aaea-41cc-8e55-3b0a7442a423,False
4,311ef0f0-47aa-4de8-aa88-12bbdfdb8ea7,1ad0536e-8422-409f-9e4a-3dc64f5e260a,False
...,...,...,...
294,390f73b5-6e10-4db6-978d-03cf06d2cc39,2cf8f595-acd3-489f-b4ca-86d02c4e1eb2,True
295,b5699703-e75a-4ea4-a8f0-1327108be3cb,3b6af836-ccfd-4dfa-83b1-937f4f62e8a1,True
296,d7009e84-c4fd-4591-abcb-124332e1746d,57ea9230-51b6-4d6f-baaf-ee06c0d78318,True
297,4904f2e0-0876-4f25-bba3-97806456bd31,3739da22-e526-43b7-a5d5-673b905f0908,True


In [243]:
dict_ = sample.set_index('uuid')['is_duplicate_for'].to_dict()

In [244]:
from tqdm import tqdm
scores = []
for duplicate, duplicate_for in tqdm(dict_.items()):
    hashes_video1 = get_video_hashes(f'train_dataset/{duplicate}.mp4', frame_interval=30)
    hashes_video2 = get_video_hashes(f'train_dataset/{duplicate_for}.mp4', frame_interval=30)
    average_distance = hamming_distance(hashes_video1, hashes_video2)
    scores.append(average_distance)

100%|██████████| 292/292 [09:59<00:00,  2.05s/it]


In [248]:
pred_labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [245]:
threshold = 5
true_labels = sample['is_duplicate'].astype(int).to_list()
pred_labels = [1 if dist <= threshold else 0 for dist in scores]
f1 = f1_score(true_labels, pred_labels)
f1

ValueError: Found input variables with inconsistent numbers of samples: [299, 292]

In [249]:
len(true_labels)

299

In [250]:
len(scores)

292